In [ ]:
import sys
from chart_studio import plotly
import plotly.graph_objs as go
import plotly.offline as pltlyoff
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Custom files ###
dir_GPPlotly = 'C:\\Users\\tln229\\Downloads\\Python Tests\\Constrained GP_2\\GP_2\\gp_plotly\\'
sys.path.append(dir_GPPlotly)

dir_gp_constr = 'C:\\Users\\tln229\\Downloads\\Python Tests\\Constrained GP_2\\GP_2\\gp_constr\\'
sys.path.append(dir_gp_constr)

# Import
from GPPlotly.plottingfunctions import PlotGP2d, add_traces_to_fig, gp_diagnostics, pred_vs_error_perc
from GPConstr.model import GPmodel, Constraint
from GPConstr.kern import kernel_Matern52

In [2]:
def get_diagnostics_df(x_test, y_test, mean, var):
    """ Put test data in a dataframe"""
    df = pd.DataFrame(x_test)
    df.columns = ['x' + str(i+1) for i in range(x_test.shape[1])]
    df['y_true'] = y_test
    df['y_mean'] = mean
    df['y_var'] = var
    
    return df

In [3]:
# ---------- DATA ----------
def func(x):
    return x[:, 0] - x[:, 1]

# training
n_train = 20
x1 = np.linspace(0, 10, n_train).reshape(-1, 1)
x2 = np.random.uniform(0, 10, n_train).reshape(-1, 1)

x_train = np.concatenate((x1, x2), axis = 1)
y_train = func(x_train) + np.random.uniform(-2, 2, n_train)

# testing
n_test = 10
x1_test = np.linspace(0, 10, n_test).reshape(-1, 1)
x2_test = np.random.uniform(0, 10, n_test).reshape(-1, 1)

x_test = np.concatenate((x1_test, x2_test), axis = 1)
y_test = func(x_test)

In [4]:
# ----- MODEL -----
ker = kernel_Matern52(variance = 1, lengthscale = [1]*x_train.shape[1])
model = GPmodel(kernel = ker, likelihood = 1, mean = 0)

# feed data to model
model.X_training = x_train
model.Y_training = y_train

In [5]:
# ----- OPTIMIZE -----
model.optimize(include_constraint = False, fix_likelihood = False)

..Running optimization for unconstrained GP ... DONE - Total time: 0.023 seconds


In [6]:
# Run unconstrained model
mean_unconstr, var_unconstr = model.calc_posterior_unconstrained(x_test, full_cov = False)
mean_unconstr = np.array(mean_unconstr).flatten()
var_unconstr = np.array(var_unconstr).flatten()

# Collect data in df
df_unconstr = get_diagnostics_df(x_test, y_test, mean_unconstr, var_unconstr)

# View first couple of records of dataframe with testing data
df_unconstr.head()

..Running calculation of K_w ... SKIP - (cached)
..Running calculation of Cholesky factor for K_w ... SKIP - (cached)
..Calculating f* | Y ... DONE - Total time: 0.000 seconds


,x1,x2,y_true,y_mean,y_var
0,0.000000,5.908102,-5.908102,-5.092416,0.260415
1,1.111111,6.241705,-5.130594,-4.674410,0.093729
2,2.222222,0.073378,2.148844,1.624589,0.083439
3,3.333333,7.431405,-4.098072,-3.145597,0.164109
4,4.444444,0.524807,3.919638,5.879811,0.447183


In [7]:
# Plot
figs = gp_diagnostics(df_unconstr, y_name = 'y', subplots = False)
pltlyoff.iplot(figs[1], filename='')